# Problema de Busca - Super Mario Bros
## Projeto da disciplina **SCC0230 - Inteligência Artificial**
### Assunto: Busca Informada e Busca Cega

## Membros

* Alexandre Eduardo de Souza Jesus - alexandre_souza@usp.br - 12559506

* Arthur Santorum Lorenzetto - arts.lorenzetto@usp.br -

* Eduardo Zaffari Monteiro - eduardozaffarimonteiro@usp.br - 

* Gustavo Silva de Oliveira - gustavo.oliveira03@usp.br - 12567231 (Vice-Líder)

* Lucas Ivars Cadima Ciziks - luciziks@usp.br - 125599472 (Líder)

* Pedro Henrique de Freitas Maçonetto - pedromaconetto@usp.br - 

* Vitor Okubo Ianella - vitorok.03@gmail.com - 

## Introdução

No mundo real, há uma infinidade de questões que podem ser classificadas como **Problemas de Busca**, tais quais a logística de entrega de um produto, o melhor caminho entre duas estações de trem e até mesmo na resolução de jogos. Nesse contexto, é importante compreender as diferentes abordagens e aplicações envolta desses problemas no estudo da Inteligência Artificial. Resumidamente, uma **Busca Cega** ou Não Informada é uma abordagem que não utiliza nenhuma informação adicional além do que definido no problema original, tentando exaustivamente até encontrar a solução ou falhar. Já a **Busca Informada** utiliza uma heurística, algum conhecimento ou informação que auxila nas decisões e torna a busca mais eficiente. Assim, o presente projeto possui como objetivo implementar e comparar os resultados entre algoritmos de Busca Cega e Busca Informada para um mesma problemática em investigação: **O Resgate da Princesa Peach**.

Nesse problema, baseado no mundo fantástico de [Super Mario Bros](https://pt.wikipedia.org/wiki/Super_Mario_Bros.#Enredo), o encanador Mario precisa resgatar a Princesa Peach, raptada pelo vilão Bowser, viajando até sua ardente fortaleza. Em seu caminho, Mario deve enfrentar e derrotar os inimigos que tentarão impedir sua passagem até o castelo. Para isso, Mario precisa comer um **Cogumelo** e assim derrotá-los para finalmente chegar ao castelo e derrotar Bowser. Abstraindo para um problema de busca, Mario começa em um ponto do labirinto (estado inicial) e, antes de chegar ao castelo (estado final), ele precisa comer o **cogumelo** (primeira condição de parada) para derrotar todos os **Goombas** (obstáculos). É importante destacar que antes de comer o cogumelo, os Goombas não permitirão a passagem de Mario.

## Modelagem do Problema

Detalhando o problema apresentado:

* **Estados**: Configuração correnta da matriz esparsa;

* **Operadores**: Mover o Mario uma casa em 4 direções possíveis (Leste, Oeste, Norte ou Sul);

* **Movimentos Válidos**: Não parede / Não Goomba antes do Cogumelo / Goomba após o Cogumelo / Cogumelo / Cogumelo Comido;

* **Configuração Final**: O Cogumelo foi comido, todos os Goombas foram derrotados e o Mario chegou ao Castelo.

* **Custo do Caminho**: Cada movimento tem um custo unitário.

Em nosso processamento, a matriz de entrada contendo o labirinto será um arquivo csv, como a do exemplo utilizado. Visualmente, ela será representada da seguinte forma:

![Exemplo do Labirinto](media/exemplo.png)

em que:
* 0 representa uma parede;
* 1 é um corredor;
* 2 é um Goomba/inimigo;
* 3 representa o ponto inicial do Mario;
* 4 é o castelo do Bowser (Ponto Final);
* 5 é o cogumelo que fornece poderes ao Mario.


## Busca Não Informada

A Busca Não Informada escolhida para resolver nosso problema foi a **Busca Cega em Profundidade** (Depth-First). Isso se deve à economia de memória proporcionada quando em comparação à busca em largura (Breadth-First). A ordem de expansão definida em nosso algoritmo foi Leste, Oeste, Norte e Sul. Para computar a matriz de entrada, transformamos-na em um **Grafo**, onde mapeamos todas as possíveis extensões/caminhos de cada nó (coordenada da matriz). Isso, em conjunto com a estrutura de dados **Pilha**, auxilia a manipulação dos dados e o *backtracking* na busca.

Uma adaptação necessária para nosso problema foi o reinício das posições visitadas (exceto pela posição atual) após o Mario consumir um cogumelo ou derrotar um Goomba.

In [22]:
import pandas as pd
import numpy as np
from collections import deque

In [23]:
# Capturando pontos especiais
def important_points(maze):    
    start_point = np.where(maze == 3)
    start = (int(start_point[0]), int(start_point[1]))
    
    end_point = np.where(maze == 4)
    end = (int(end_point[0]),int(end_point[1]))
    
    mushroom_point = np.where(maze == 5)
    mushroom = (int(mushroom_point[0]),int(mushroom_point[1]))
    
    goomba_points = np.where(maze == 2)
    goombas_x = goomba_points[0]
    goombas_y = goomba_points[1]
    goombas = []
    for i in range(len(goombas_x)):
        goombas.append((int(goombas_x[i]), int(goombas_y[i])))

        # Sem cogumelo, Goomba é interpretado como uma parede
        maze[int(goombas_x[i])][int(goombas_y[i])] = 0
    
    maze[start_point] = 1
    maze[end_point] = 1
    
    return start, end, mushroom, goombas

In [24]:
# Função responsável por transformar labirinto em grafo de pontos
def maze2graph(maze):
    colunas = len(maze)
    linhas = len(maze[0])
    grafo = {(i, j): [] for j in range(linhas) for i in range(colunas) if maze[i][j]}
    
    for linha, coluna in grafo.keys():
        
        if not(linha < colunas - 1 and not maze[linha + 1][coluna]):
            grafo[(linha, coluna)].append(("S", (linha + 1, coluna)))
            grafo[(linha + 1, coluna)].append(("N", (linha, coluna)))

        if not(coluna < linhas - 1 and not maze[linha][coluna + 1]):
            grafo[(linha, coluna)].append(("L", (linha, coluna + 1)))
            grafo[(linha, coluna + 1)].append(("O", (linha, coluna)))

    return grafo

In [25]:
# Implementação da Busca Cega em Profundidade
def depth_first_algorithm(maze, start, end):
    pilha = deque([("", start)])
    visitado = set()
    graph = maze2graph(maze)
    
    while pilha:
        caminho, atual = pilha.pop()
        if atual == end:
            return caminho

        if atual in visitado:
            continue

        visitado.add(atual)
        for direcao, vizinho in graph[atual]:
            pilha.append((caminho + direcao, vizinho))
            
    return "Sem caminho!"

In [26]:
# Insere o caminho encontrado na matriz do labirinto
def visualize_path(maze, path, start):
    caminho_atual = start
    path = path[0:len(path)]

    for value in list(path):

        if value == 'L':
            maze[caminho_atual[0]][caminho_atual[1]] = '→'
            caminho_atual = (caminho_atual[0], caminho_atual[1] + 1)

        elif value == 'O':
            maze[caminho_atual[0]][caminho_atual[1]] = '←'
            caminho_atual = (caminho_atual[0], caminho_atual[1] - 1)

        elif value == 'N':
            maze[caminho_atual[0]][caminho_atual[1]] = '↑'
            caminho_atual = (caminho_atual[0] - 1, caminho_atual[1])

        elif value == 'S':
            maze[caminho_atual[0]][caminho_atual[1]] = '↓'
            caminho_atual = (caminho_atual[0] + 1, caminho_atual[1])
        
    return maze

In [27]:
# Função responsável por copiar o labirinto
def copy_maze(maze, start, end, mushroom, goombas):
    maze_copy = maze.copy().tolist()
    maze_copy[start[0]][start[1]] = 'I'
    maze_copy[end[0]][end[1]] = 'F'
    maze_copy[mushroom[0]][mushroom[1]] = 'C'
    for goomba in goombas:
        maze_copy[goomba[0]][goomba[1]] = 'G'
    maze_copy = [[str(ele) for ele in sub] for sub in maze_copy]
    
    return maze_copy

In [28]:
# Função responsável por solucionar o labirinto
def solved_maze(maze):
    # Recuperando pontos especiais
    start, end, mushroom, goombas = important_points(maze)

    # Encontrando o caminho entre o ponto inicial e o cogumelo (primeira condição de parada)
    start_to_mushroom = depth_first_algorithm(maze, start, mushroom)

    # Agora o Mario tem poder para derrotar o Goomba
    for goomba in goombas:
        maze[goomba[0]][goomba[1]] = 1

    # Encontrando o caminho entre o cogumelo e o goomba
    mushroom_to_goomba_paths = []
    start_goomba = mushroom
    for next_goomba in goombas:
        mushroom_to_goomba_paths.append(depth_first_algorithm(maze, start_goomba, next_goomba))
        start_goomba = next_goomba
    
    # Encontrando o caminho entre o goomba e o ponto final
    goomba_to_end = depth_first_algorithm(maze, start_goomba, end)

    # Visualizando caminhos
    start_mushroom_matrix = visualize_path(copy_maze(maze, start, end, mushroom, goombas), start_to_mushroom, start)

    mushroom_goomba_matrixes = []
    start_goomba = mushroom
    for i in range(len(goombas)):
        mushroom_goomba_matrixes.append(visualize_path(copy_maze(maze, start, end, mushroom, goombas), mushroom_to_goomba_paths[i], start_goomba))
        start_goomba = goombas[i]
        
    goomba_end_matrix = visualize_path(copy_maze(maze, start, end, mushroom, goombas), goomba_to_end, start_goomba)

    return start_mushroom_matrix, *mushroom_goomba_matrixes, goomba_end_matrix

In [29]:
# Lendo Labirinto do arquivo CSV
maze_csv = pd.read_csv('mazes/maze_easy.csv')

# Transformando Labirinto em matriz completa
maze = maze_csv.to_numpy()

print(maze)

# Resolvendo o labirinto
start_mushroom_matrix, *mushroom_goomba_matrixes, goomba_end_matrix = solved_maze(maze)

[[0 0 0 0 0 0 0]
 [0 3 1 1 2 1 0]
 [0 1 0 0 0 1 0]
 [0 1 0 5 1 2 0]
 [0 1 1 1 0 4 0]
 [0 0 0 0 0 0 0]]


In [30]:
start_mushroom_matrix

[['0', '0', '0', '0', '0', '0', '0'],
 ['0', '↓', '1', '1', 'G', '1', '0'],
 ['0', '↓', '0', '0', '0', '1', '0'],
 ['0', '↓', '0', 'C', '1', 'G', '0'],
 ['0', '→', '→', '↑', '0', 'F', '0'],
 ['0', '0', '0', '0', '0', '0', '0']]

In [31]:
mushroom_goomba_matrixes

[[['0', '0', '0', '0', '0', '0', '0'],
  ['0', 'I', '1', '1', 'G', '←', '0'],
  ['0', '1', '0', '0', '0', '↑', '0'],
  ['0', '1', '0', '→', '→', '↑', '0'],
  ['0', '1', '1', '1', '0', 'F', '0'],
  ['0', '0', '0', '0', '0', '0', '0']],
 [['0', '0', '0', '0', '0', '0', '0'],
  ['0', 'I', '1', '1', '→', '↓', '0'],
  ['0', '1', '0', '0', '0', '↓', '0'],
  ['0', '1', '0', 'C', '1', 'G', '0'],
  ['0', '1', '1', '1', '0', 'F', '0'],
  ['0', '0', '0', '0', '0', '0', '0']]]

In [32]:
goomba_end_matrix

[['0', '0', '0', '0', '0', '0', '0'],
 ['0', 'I', '1', '1', 'G', '1', '0'],
 ['0', '1', '0', '0', '0', '1', '0'],
 ['0', '1', '0', 'C', '1', '↓', '0'],
 ['0', '1', '1', '1', '0', 'F', '0'],
 ['0', '0', '0', '0', '0', '0', '0']]

## Busca Informada

* Justificar escolha do algoritmo;
* Fundamentar heurística utilizada (Admissibilidade);
* Especificar decisões de implementação: Estruturas de Dados, manipulação dos dados, adaptações ao problema, etc.

In [33]:
# Implementar a Busca Informada do Trabalho
# A*: https://www.geeksforgeeks.org/a-search-algorithm/
# Greedy Best-First: https://www.geeksforgeeks.org/best-first-search-informed-search/


In [34]:
maze = pd.read_csv('mazes/maze_hard.csv',header=None).values

In [35]:
maze

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 1, 1, 1, 3, 1, 1, 1, 0],
       [0, 1, 0, 1, 0, 1, 0, 0, 4, 0],
       [0, 1, 0, 6, 0, 1, 0, 0, 1, 0],
       [0, 1, 0, 0, 0, 1, 0, 1, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 5, 0],
       [0, 1, 1, 1, 1, 1, 1, 1, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int64)

In [36]:
def maze2graph(maze):
    height = len(maze)
    width = len(maze[0]) if height else 0
    graph = {(i, j): [] for j in range(width) for i in range(height) if maze[i][j]}
    for row, col in graph.keys():
        if not(row < height - 1 and not maze[row + 1][col]):
            graph[(row, col)].append(("S", (row + 1, col)))
            graph[(row + 1, col)].append(("N", (row, col)))
        if not(col < width - 1 and not maze[row][col + 1]):
            graph[(row, col)].append(("L", (row, col + 1)))
            graph[(row, col + 1)].append(("O", (row, col)))
    return graph
grafo = maze2graph(maze)
grafo

{(2, 1): [('S', (3, 1)), ('L', (2, 2))],
 (3, 1): [('N', (2, 1)), ('S', (4, 1))],
 (4, 1): [('N', (3, 1)), ('S', (5, 1))],
 (5, 1): [('N', (4, 1)), ('S', (6, 1))],
 (6, 1): [('N', (5, 1)), ('S', (7, 1))],
 (7, 1): [('N', (6, 1)), ('L', (7, 2))],
 (2, 2): [('O', (2, 1)), ('L', (2, 3))],
 (7, 2): [('O', (7, 1)), ('L', (7, 3))],
 (2, 3): [('O', (2, 2)), ('S', (3, 3)), ('L', (2, 4))],
 (3, 3): [('N', (2, 3)), ('S', (4, 3))],
 (4, 3): [('N', (3, 3))],
 (7, 3): [('O', (7, 2)), ('L', (7, 4))],
 (2, 4): [('O', (2, 3)), ('L', (2, 5))],
 (7, 4): [('O', (7, 3)), ('L', (7, 5))],
 (2, 5): [('O', (2, 4)), ('S', (3, 5)), ('L', (2, 6))],
 (3, 5): [('N', (2, 5)), ('S', (4, 5))],
 (4, 5): [('N', (3, 5)), ('S', (5, 5))],
 (5, 5): [('N', (4, 5))],
 (7, 5): [('O', (7, 4)), ('L', (7, 6))],
 (2, 6): [('O', (2, 5)), ('L', (2, 7))],
 (7, 6): [('O', (7, 5)), ('L', (7, 7))],
 (2, 7): [('O', (2, 6)), ('L', (2, 8))],
 (5, 7): [],
 (7, 7): [('O', (7, 6)), ('L', (7, 8))],
 (2, 8): [('O', (2, 7)), ('S', (3, 8))],
 (3

In [37]:
#definindo heuristica manhattan
def manhattan(ponto1, ponto2):
    x_1,y_1 = ponto1
    x_2,y_2 = ponto2
    return np.abs(x_1 - x_2) + np.abs(y_1 - y_2)

In [38]:
def a_search(arad, bucharest, grafo):
    cp = [[[arad], manhattan(arad,bucharest)]] #caminhos possiveis
    atual = arad
    
    while atual != bucharest:
        caminho = cp[0][0]
        percorridos = len(caminho)
        
        for cv in grafo[caminho[-1][1]]:
            direcao, coord = cv #cv cidade vizinha
            
            if coord not in caminho:
                nc = caminho.copy()
                nc.append((direcao,coord))
                cp.append([nc, manhattan(coord,bucharest) + percorridos])
                
        cp.pop(0)
        cp.sort(key=lambda x: x[1])
        atual = cp[0][0][-1]
    return cp[0][0]

In [39]:
def mata_goomba(goombas, estrela, grafo):
    goombas.sort(key=lambda x: manhattan(x,estrela))
    cg = [a_search(estrela,goombas[0],grafo)]
    for i in range(len(goombas)-1):
        goomba1 = goombas.pop(0)
        goombas.sort(key=lambda x: manhattan(x,goomba1))
        cg.append(a_search(goomba1,goombas[0],grafo))
    return cg, goombas[0]

In [40]:
ponto_inicial = (1,5)
ponto_final = (3,3)
ponto_estrela = (5,8)
goombas = [(2,8),(4,6)]

def resolve_labirinto(ponto_inicial, ponto_final, ponto_estrela, goombas,grafo):
    inicial_estrela = a_search(ponto_inicial,ponto_estrela,grafo)
    
    for i in goombas:
        maze[i[0]][i[1]] = 1
        
    grafo = maze2graph(maze)
    
    cg, ultimo_goomba = mata_goomba(goombas, ponto_estrela, grafo)
    cgf = a_search(ultimo_goomba,ponto_final,grafo)
    return inicial_estrela,cg,cgf

In [41]:
# fé
resolve_labirinto(ponto_inicial,ponto_final,ponto_estrela,goombas,grafo)

KeyError: 5

## Conclusão

* Comparar algoritmos com alguns casos de Teste;
* Interpretar resultados obtidos;
* Considerações Finais acerca do problema.

## Referências


BRATKO, I. Prolog Programming for Artificial Intelligence . Mitchell, T. Machine
Learning , McGraw Hill, 1997.

RUSSEL, S.; NORVIG, P. Artificial Intelligence : A Modern Approach. Prentice
Hall; 3 edition (December 11, 2009).

https://www.geeksforgeeks.org/difference-between-informed-and-uninformed-search-in-ai/?ref=lbp

https://www.sciencedirect.com/topics/mathematics/manhattan-distance

https://www.geeksforgeeks.org/difference-between-bfs-and-dfs/?ref=lbp

http://theory.stanford.edu/~amitp/GameProgramming/AStarComparison.html

http://theory.stanford.edu/~amitp/GameProgramming/